In [1]:
import sys
sys.path.append('..')
from api.api import PDTB
from constituent import Constituent
import itertools
import copy
from collections import Counter

In [2]:
pdtb2 = '/home/pengfei/data/pdtb_v2/all/conll/'
pdtb3 = PDTB()

## only for one individual sentence for explicit

In [1]:
def get_constituents(docid, sentid, conn_index):
    """
    Args:
            docid(str)
            sentid(int)
            conn_index(list[int])
    Returns:
            [ [(token_index, text),...], ...]"""
    syntax_tree = pdtb3.get_syntax_tree(docid, sentid)
    if syntax_tree.tree == None:
        return []
    all_leaves = syntax_tree.tree.get_leaves()
    conn_indices = conn_index
    constituent_nodes = []
    if len(conn_indices) == 1:# like and or so...
        conn_node = syntax_tree.get_leaf_node_by_token_index(conn_indices[0]).up
    else:
        conn_node = syntax_tree.get_common_ancestor_by_token_indices(conn_indices)
        conn_leaves = set([all_leaves.index(syntax_tree.get_leaf_node_by_token_index(conn_index)) for conn_index in conn_indices])
        children = conn_node.get_children()
        for child in children:
            leaves = set([all_leaves.index(n) for n in child.get_leaves()])
            if list(leaves-conn_leaves)!=[]: constituent_nodes.append(list(leaves-conn_leaves))
    curr = conn_node
    while not curr.is_root():
        sibs = [ [all_leaves.index(n) for n in sib] for sib in syntax_tree.get_siblings(curr)]
        constituent_nodes.extend(sibs)
        curr = curr.up

    tokens_indices_with_text = pdtb3.get_tokens_indices_with_text(docid, sentid)
    subtree_list = [ [(o,tokens_indices_with_text[o][2]) for o in node] for node in constituent_nodes]
    return subtree_list

def merge_consti(subtree_list):
    return {(o[0][0],o[-1][0]):i for i,o in enumerate(subtree_list)}

def arg_is_contained(i, arg):
    docid = pdtb3.parse_data[i]['DocID']
    sentid = pdtb3.get_arg_sent_id(i, arg)
    conn_indices = [o[1] for o in pdtb3.get_arg_token_list(i, 'Connective')]
    subtree_list = get_constituents(docid, sentid[0], conn_indices)
    merged_consti_list = merge_consti(subtree_list)
    token_list = pdtb3.get_arg_token_list(i, arg)
    start,end = token_list[0][1], token_list[-1][1]
    # filter results
    consti_list = [k for k in merged_consti_list.keys() if k[0]>=start and k[1]<=end]
    # generate results
    results = []
    for i in range(1, len(consti_list)+1):
        for subset in itertools.combinations(consti_list, i):
            results.append(expand(subset))
    # check
    return [o[1] for o in token_list] in results

def expand(subset):
    ret = []
    for subsubset in subset:
        for i in range(subsubset[0], subsubset[1]+1):
            ret.append(i)
    return sorted(ret)

def get_subtree_length(i, arg):
    docid = pdtb3.parse_data[i]['DocID']
    sentid = pdtb3.get_arg_sent_id(i, arg)
    conn_indices = [o[1] for o in pdtb3.get_arg_token_list(i, 'Connective')]
    subtree_list = get_constituents(docid, sentid[0], conn_indices)
    return len(subtree_list)

In [17]:
bad = 0
total =0
for i in range(len(pdtb3.parse_data_test)):
    if pdtb3.parse_data[i]['Type'] == 'Explicit' and pdtb3.get_arg_sent_id(i,'Arg1')[0] == pdtb3.get_arg_sent_id(i,'Arg2')[0]:
        total+=1
        if not arg_is_contained(i, 'Arg1'):
            bad += 1
#             print(pdtb3.get_highlighted_parsetree(i, v=True))
print('arg1 cover rate ', 1- bad/total)


bad = 0
total =0
for i in range(len(pdtb3.parse_data_test)):
    if pdtb3.parse_data[i]['Type'] == 'Explicit' and pdtb3.get_arg_sent_id(i,'Arg1')[0] == pdtb3.get_arg_sent_id(i,'Arg2')[0]:
        total+=1
        if not arg_is_contained(i, 'Arg2'):
            bad += 1
#             print(pdtb3.get_highlighted_parsetree(i, v=True))
print('arg2 cover rate ', 1- bad/total)

arg1 cover rate  0.8715415019762845
arg2 cover rate  0.9189723320158103


In [14]:
length = 0
total =0
for i in range(len(pdtb3.parse_data_test)):
    if pdtb3.parse_data[i]['Type'] == 'Explicit' and pdtb3.get_arg_sent_id(i,'Arg1')[0] == pdtb3.get_arg_sent_id(i,'Arg2')[0]:
        total+=1
        length += get_subtree_length(i, 'Arg1')
print('arg1', length/total)


length = 0
total =0
for i in range(len(pdtb3.parse_data_test)):
    if pdtb3.parse_data[i]['Type'] == 'Explicit' and pdtb3.get_arg_sent_id(i,'Arg1')[0] == pdtb3.get_arg_sent_id(i,'Arg2')[0]:
        total+=1
        length += get_subtree_length(i, 'Arg2')
print('arg2', length/total)

arg1 7.988142292490118
arg2 7.988142292490118


In [107]:
# def arg_clauses(docid, sentid):
#     token_indices_with_text = pdtb3.get_tokens_indices_with_text(docid, sentid)
#     sent_tokens = [(i,t) for _,i,t in token_indices_with_text]
#     # step 1: split the word
#     punctuation = "...,:;?!~--"
#     _clause_indices_list = []#[[(1,"I")..], ..]
#     temp = []
#     for index, word in sent_tokens:
#         if word not in punctuation:
#             temp.append((index, word))
#         else:
#             if temp != []:
#                 _clause_indices_list.append(temp)
#                 temp = []
#     if temp != []: _clause_indices_list.append(temp)
#     # strip punctuations in the start or end 
#     clause_indices_list = []
#     for clause_indices in _clause_indices_list:
#         temp = list_strip_punctuation(clause_indices)
#         if temp != []:
#             clause_indices_list.append([item[0] for item in temp])

#     # step2: then use SBAR tag in its parse tree to split each part into clauses.
#     syntax_tree = pdtb3.get_syntax_tree(docid, sentid)
#     if syntax_tree.tree == None:
#         return []
#     clause_list = []
#     for clause_indices in clause_indices_list:
#         clause_tree = _get_subtree(syntax_tree, clause_indices)
#         flag = 0
#         for node in clause_tree.tree.traverse(strategy="levelorder"):
#             if node.name == "SBAR":
#                 temp1 = [node.index for node in node.get_leaves()]
#                 temp2 = sorted(list(set(clause_indices) - set(temp1)))

#                 if temp2 == []:
#                     clause_list.append(temp1)
#                 else:
#                     if temp1[0] < temp2 [0]:
#                         clause_list.append(temp1)
#                         clause_list.append(temp2)
#                     else:
#                         clause_list.append(temp2)
#                         clause_list.append(temp1)
#                 flag = 1
#                 break
#         if flag == 0:
#             clause_list.append(clause_indices)
#     clauses = []
#     for clause_indices in clause_list:
#         clauses.append(clause_indices)
#     return [ [(o,token_indices_with_text[o][2]) for o in c] for c in clauses]

# def list_strip_punctuation(list):
#     punctuation = """!"#&'*+,-..../:;<=>?@[\]^_`|~""" + "``" + "''"
#     i = 0
#     while i < len(list) and list[i][1] in punctuation + "-LCB--LRB-":
#         i += 1
#     if i == len(list):
#         return []
#     j = len(list) - 1
#     while j >= 0 and list[j][1] in punctuation + "-RRB--RCB-":
#         j -= 1
#     return list[i: j+1]

# def _get_subtree(syntax_tree, clause_indices):
#     copy_tree = copy.deepcopy(syntax_tree)

#     for index, leaf in enumerate(copy_tree.tree.get_leaves()):
#         leaf.add_feature("index",index)

#     clause_nodes = []
#     for index in clause_indices:
#         node = copy_tree.get_leaf_node_by_token_index(index)
#         clause_nodes.append(node)

#     for node in copy_tree.tree.traverse(strategy="levelorder"):
#         node_leaves = node.get_leaves()
#         if set(node_leaves) & set(clause_nodes) == set([]):
#             node.detach()
#     return copy_tree


# def merge_consti(subtree_list):
#     return {(o[0][0],o[-1][0]):i for i,o in enumerate(subtree_list)}

# def expand(subset):
#     ret = []
#     for subsubset in subset:
#         for i in range(subsubset[0], subsubset[1]+1):
#             ret.append(i)
#     return sorted(ret)

# def get_subtree_length(i, arg):
#     docid = pdtb3.parse_data[i]['DocID']
#     sentid = pdtb3.get_arg_sent_id(i, arg)
#     conn_indices = [o[1] for o in pdtb3.get_arg_token_list(i, 'Connective')]
#     subtree_list = get_constituents(docid, sentid[0], conn_indices)
#     return len(subtree_list)


# def arg_is_contained_clause(i, arg):
#     docid = pdtb3.parse_data[i]['DocID']
#     sentids = pdtb3.get_arg_sent_id(i, arg)
#     clause_list = arg_clauses(docid, sentid[0])
#     merged_clause_list = merge_consti(clause_list)
#     token_list = pdtb3.get_arg_token_list(i, arg)
#     token_text = pdtb3.get_tokens_text(docid, token_list)
#     assert(len(token_list) == len(token_text))
#     start,end = token_list[0][1], token_list[-1][1]
#     # filter results
#     clause_list = [k for k in merged_clause_list.keys() if k[0]>=start and k[1]<=end]
#     # generate results
#     results = []
#     for i in range(1, len(clause_list)+1):
#         for subset in itertools.combinations(clause_list, i):
#             results.append(expand(subset))
#     # check
#     return contained([o[1] for o in token_list], results, token_text)

# def contained(token_list, results, token_text):
#     punctuation = """!"#&'*+,-..../:;<=>?@[\]^_`|~""" + "``" + "''"
#     for result in results:
#         remain = set(token_list).symmetric_difference(set(result))
#         whether = True
#         for r in remain:
#             if r not in token_list: continue
#             if token_text[token_list.index(r)] not in punctuation:
#                 whether = False
#         if whether: return True
#     return False

# only for one individual sentence for implicit
# for PS case for explicit
# for PS case for implicit

In [119]:
a = []
total = 10000
for i in range(total):
    if not (pdtb3.parse_data[i]['Type'] == 'Explicit' and pdtb3.get_arg_sent_id(i,'Arg1')[0] == pdtb3.get_arg_sent_id(i,'Arg2')[0]):
        a.append(pdtb3.arg_is_contained_clause(i, 'Arg1'))
dis = Counter(a)
print({k:v/len(a) for k,v in dis.items()})
print({k:v/total for k,v in dis.items()})

{False: 0.12113295751409323, True: 0.8788670424859067}
{False: 0.0881, True: 0.6392}


In [118]:
a = []
total = 10000
for i in range(total):
    if not (pdtb3.parse_data[i]['Type'] == 'Explicit' and pdtb3.get_arg_sent_id(i,'Arg1')[0] == pdtb3.get_arg_sent_id(i,'Arg2')[0]):
        a.append(pdtb3.arg_is_contained_clause(i, 'Arg2'))
dis = Counter(a)
print({k:v/len(a) for k,v in dis.items()})
print({k:v/total for k,v in dis.items()})

{True: 0.8222191667812457, False: 0.1777808332187543}
{True: 0.598, False: 0.1293}
